Instruction:
1. get token and accept license (two links below)
2. in menu above press Runtime - Run all
3. wait for link `nnn.gradio.app` in section *Actual launch of WebUI server*
4. use *Login* and *Pass* from `remote_login` and `remote_pass` fields below

Look *Useful links* below.

In [ ]:
#@title Parameters

#@markdown # Token and atleast one link is Requied!

#@markdown get a token from https://huggingface.co/settings/tokens
huggingface_user_token = "" #@param {type:"string"}

#@markdown #Set atleast one model:

#@markdown model to load
use_model = 'Model 1' #@param ["Model 1", "Model 2", "Model 3", "Model 4", "Model 5", "Model 6"]

#@markdown model URL to download, just to save URL and not type every time

#@markdown ___
#@markdown **Default SD 1.4 model**, require accepting license:\
#@markdown accept license there https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/tree/main to use this model

model_1_url = "https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt" #@param {type:"string"}

#@markdown ___
#@markdown **WaifuDiffusion 1.3 model** (warning it may generate nsfw, dont do it on Google Colab):
model_2_url = "https://huggingface.co/hakurei/waifu-diffusion-v1-3/resolve/main/wd-v1-3-float32.ckpt" #@param {type:"string"}

#@markdown ___
#@markdown **SD 1.5 model** 4Gb emaonly
model_3_url = "https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt" #@param {type:"string"}

#@markdown **SD 1.5 model** 8Gb
model_4_url = "https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned.ckpt" #@param {type:"string"}

#@markdown ___
#@markdown **Openjourney model** is an open source Stable Diffusion fine tuned model on Midjourney images.\
#@markdown (just add **`mdjrny-v4 style`** at the beginning of prompt)
model_5_url = "https://huggingface.co/prompthero/openjourney/resolve/main/mdjrny-v4.ckpt" #@param {type:"string"}

#@markdown ___
#@markdown **Complex-Lineart model**\
#@markdown Use prompt: `ComplexLA style`\
#@markdown Use resolution near 768x768, lower resolution works but quality will not be as good.
model_6_url = "https://huggingface.co/Conflictx/Complex-Lineart/resolve/main/ComplexLA%20Style.ckpt" #@param {type:"string"}

#@markdown ___
#@markdown #WebUI only, look **after launching Actual launch** section to get link

remote_login = "me" #@param {type:"string"}

remote_pass = "test" #@param {type:"string"}

#@markdown # Next happening after manual stoppind WebUI in Collab (by pressing Stop Cell)

#@markdown Upload everything generated to yours Google drive
save_to_Google_drive_at_end = 'false' #@param ["true", "false"]

#@markdown will show all generated images on this Collab page, after stoping WebUI
show_all_generated_images_at_end = 'false' #@param ["true", "false"]



# **Remember to save results** after finishing generation of images! (**bottom**) 


# Useful links:

https://lexica.art/ - images with prompts

https://huggingface.co/Gustavosta/MagicPrompt-Stable-Diffusion?text=in+space+with - text model trained on lexica art above, it can generate prompts - look at right side there *Text Generation*, type your text there.

https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Features - Features of SD


In [ ]:
!uname -a
!nvidia-smi

In [ ]:
import os
import re
from urllib.parse import unquote
models_urls = [model_1_url, model_2_url, model_3_url, model_4_url, model_5_url, model_6_url]
model_index = 0 if use_model == 'Model 1' else 1 if use_model == 'Model 2' else 2 if use_model == 'Model 3' else 3 if use_model == 'Model 4' else 4 if use_model == 'Model 5' else 5
local_model_url = models_urls[model_index]
local_model_url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', local_model_url)[0]
if(len(local_model_url)<6):
  raise TypeError("model_url is not correct")
local_file_name = os.path.basename(local_model_url).split('/')[-1]
local_file_name = unquote(local_file_name)
if(len(local_file_name)<4):
  raise TypeError("file name in model_url is not correct")

Clone webui repository

In [ ]:
if(not os.path.exists("/content/stable-diffusion-webui")):
  !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
%cd stable-diffusion-webui

Download the model from huggingface. 

In [ ]:
#@title Model
user_header = f"\"Authorization: Bearer {huggingface_user_token}\""
tflname = "/content/stable-diffusion-webui/"+local_file_name
if(not os.path.exists(tflname)):
  !wget --header={user_header} $local_model_url
if(os.path.exists("/content/stable-diffusion-webui/model.ckpt") or os.path.islink("/content/stable-diffusion-webui/model.ckpt")):
  !rm "/content/stable-diffusion-webui/model.ckpt"
!ln -s "$tflname" "/content/stable-diffusion-webui/model.ckpt"

if(not os.path.exists("/content/stable-diffusion-webui/model.ckpt")):
  raise FileNotFoundError("error during creation of model.ckpt")

In [ ]:
!COMMANDLINE_ARGS="--exit" REQS_FILE="requirements.txt" python launch.py

Change into Web UI directory and download updates

In [ ]:
%cd /content/stable-diffusion-webui
!git pull

# **Actual launch** of WebUI server.

---
Launch web ui. You will get a link to nnn.gradio.app, follow it.

Commandline arguments are:
  - `--share` - create online gradio.app link
  - `--gradio-debug` - print outputs to console
  - `--gradio-auth username:pass` - add authentication to gradio: username, password

In [ ]:
!COMMANDLINE_ARGS="--share --gradio-debug --gradio-auth $remote_login:$remote_pass" REQS_FILE="requirements.txt" python launch.py

commands for ***after*** you have gotten done with a session
============================================================================

Zip images for downloading on local drive (click the folder icon on the left, the one below {x})

In [ ]:
if(save_to_Google_drive_at_end=="true"):
  # find does rename to fix names to ASCII
  !find /content/stable-diffusion-webui/ -type f -exec bash -c 'for f do d=${f%/*} b=${f##*/} nb=${b//[^A-Za-z0-9._-]/_}; [[ $b = "$nb" ]] || mv "$f" "$d/$nb"; done' _ {} +
  !zip -r /content/stable-diffusion-webui /content/stable-diffusion-webui/outputs 

Save images to Google Drive **Warning: this will cause google to scan your drive, so if you intend to use this and worry about that kind of stuff, probablly just set this up on a clean account that's just for this colab**

In [ ]:
from google.colab import drive

if(save_to_Google_drive_at_end=="true"):
  drive_path = "/content/drive"
  drive.mount(drive_path,force_remount=False)
  save_dir = "/content/drive/MyDrive/Stable_Dif_m_"+local_file_name+"_res/"
  !mkdir $save_dir
  !cp -r "/content/stable-diffusion-webui/outputs" $save_dir
  print("Copyng to Google Drive started, this is not instant, for many files you need to wait when all files appears in your Google Drive")


Display images on this Collab page

In [ ]:

from IPython.display import clear_output
import matplotlib.pyplot as plt
from time import sleep
folders_to_show = ['img2img-grids', 'img2img-images', 'txt2img-grids', 'txt2img-images', ]
ext_to_show = ['png', 'jpg', 'jpeg']
out_path_dir = '/content/stable-diffusion-webui/outputs/'
if(show_all_generated_images_at_end=="true"):
  for folder_name in folders_to_show:
    imdir = out_path_dir+folder_name+"/"
    img_files = []
    [img_files.extend(glob.glob(imdir + '*.' + e)) for e in ext_to_show]
    print("Folder "+folder_name+" content:")
    for img_filename in img_files:
      try:
        img = plt.imread(img_filename)
        plt.imshow(img)
        plt.show()
      except:
        print("Error reading file - " + img_filename)
      sleep(1)